In [1]:
!pip install gradio -q

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.5 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from ultralytics import YOLO
import cv2
import numpy as np

try:
    model = YOLO('best_float32.tflite')
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please make sure 'best_float32.tflite' is uploaded to your Colab session.")

try:
    with open('labels.txt', 'r') as f:
        class_names = [line.strip() for line in f.readlines()]
except FileNotFoundError:
    print("Error: 'labels.txt' not found. Please upload the labels file.")
    class_names = []

color_map = {
    "helmet": (0, 255, 0),
    "head": (0, 0, 255),
    "person": (255, 0, 0)
}

def predict_image(image):
    """
    Runs YOLOv8 TFLite inference and manually draws bounding boxes.
    """
    img_to_draw = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    results = model.predict(source=image, conf=0.25)
    result = results[0]

    for box in result.boxes:
        coords = box.xyxy[0].cpu().numpy().astype(int)
        class_id = int(box.cls[0].cpu().numpy())
        confidence = box.conf[0].cpu().numpy()

        class_name = class_names[class_id]
        color = color_map.get(class_name, (255, 255, 255))

        x1, y1, x2, y2 = coords
        cv2.rectangle(img_to_draw, (x1, y1), (x2, y2), color, 2)

        label = f"{class_name} {confidence:.2f}"
        (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(img_to_draw, (x1, y1 - h - 10), (x1 + w, y1), color, -1)
        cv2.putText(img_to_draw, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    annotated_image_rgb = cv2.cvtColor(img_to_draw, cv2.COLOR_BGR2RGB)

    return annotated_image_rgb

with gr.Blocks() as iface:
    gr.Markdown("# ⛑️ Hard Hat Detection (TFLite Model)")
    gr.Markdown("Upload an image to detect helmets (green), heads without helmets (red), and people (blue).")

    with gr.Row():
        image_input = gr.Image(type="numpy", label="Upload Image")
        image_output = gr.Image(type="numpy", label="Detected Objects")

    btn = gr.Button("Detect Objects")
    btn.click(fn=predict_image, inputs=image_input, outputs=image_output)

iface.launch(share=True, debug=True)

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1cf4a319006996be63.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Loading best_float32.tflite for TensorFlow Lite inference...

0: 640x640 4 helmets, 132.5ms
Speed: 7.1ms preprocess, 132.5ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call_


0: 640x640 1 helmet, 1 head, 183.8ms
Speed: 7.4ms preprocess, 183.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
